In [2]:
from vizdoom import *
import random 
import time

In [ ]:
from vizdoom import *
import random
import time

game = DoomGame()
game.load_config("vizdoom/scenarios/basic.cfg")
game.init()
game.set_render_crosshair(True)

shoot = [0, 0, 1]
left = [1, 0, 0]
right = [0, 1, 0]
actions = [shoot, left, right]

episodes = 10
for i in range(episodes):
    game.new_episode()
    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        misc = state.game_variables
        reward = game.make_action(random.choice(actions))
        print( "\treward:", reward)
        time.sleep(0.02)
    print ("Result:", game.get_total_reward())
    time.sleep(2)

	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: 100.0
	reward: -1.0
Result: 94.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: 100.0
	reward: -1.0
Result: 92.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -6.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -6.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -1.0
	reward: -6.0
	rewar

## Going to use OpenAI Gym for this
There is a library already for wrapping VizDoom in Gym. However, I want to learn how to wrap environments in OpenAI Gym myself for the future
 

In [3]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import cv2
import numpy as np
from vizdoom import *
import random 
import time

In [ ]:
from vizdoom import *
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

class VizDoomEnv(gym.Env):
    def __init__(self, config_file, render=False):
        super(VizDoomEnv, self).__init__()
        self.game = DoomGame()
        self.game.load_config(config_file)
        self.game.set_window_visible(render)
        self.game.set_mode(Mode.PLAYER)
        self.game.init()

        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.actions = np.identity(3, dtype=int).tolist()  # Define the actions
        self.action_space = spaces.Discrete(len(self.actions))  # TURN_LEFT, TURN_RIGHT, ATTACK
        # Example for using image as input:
        self.observation_space = spaces.Box(low=0, high=255, shape=(240, 320, 3), dtype=np.uint8)

    def step(self, action):
        self.game.make_action(self.actions[action])  # Use the defined actions
        state = self.game.get_state()
        reward = self.game.get_last_reward()
        done = self.game.is_episode_finished()
        info = {}
        if state is not None and state.screen_buffer is not None:
            observation = state.screen_buffer.transpose(1, 2, 0)  # Transpose to (240, 320, 3)
        else:
            observation = np.zeros(self.observation_space.shape, dtype=np.uint8)
        terminated = done
        truncated = False  # Set this to True if you have a condition for truncation
        return observation, reward, terminated, truncated, info

    def reset(self, **kwargs):
        # Handle the seed argument if provided
        seed = kwargs.get('seed', None)
        if seed is not None:
            self.game.set_seed(seed)
        self.game.new_episode()
        state = self.game.get_state()
        if state is not None and state.screen_buffer is not None:
            observation = state.screen_buffer.transpose(1, 2, 0)  # Transpose to (240, 320, 3)
        else:
            observation = np.zeros(self.observation_space.shape, dtype=np.uint8)
        return observation, {}

    def render(self, mode='human'):
        pass

    def close(self):
        self.game.close()

In [30]:
# test
env = ViZDoomGame(render=True)

In [26]:
state = env.reset()

In [28]:
env.step(0)

(array([[[67, 67, 67, ..., 67, 67, 67],
         [91, 91, 79, ..., 91, 79, 79],
         [67, 67, 67, ..., 67, 67, 67],
         ...,
         [19, 19, 11, ..., 47, 47, 55],
         [19, 27, 19, ..., 47, 47, 47],
         [11, 19, 19, ..., 27, 19, 19]],
 
        [[67, 67, 67, ..., 67, 67, 67],
         [91, 91, 79, ..., 91, 79, 79],
         [67, 67, 67, ..., 67, 67, 67],
         ...,
         [19, 19, 11, ..., 47, 47, 55],
         [19, 27, 19, ..., 47, 47, 47],
         [11, 19, 19, ..., 27, 19, 19]],
 
        [[67, 67, 67, ..., 67, 67, 67],
         [91, 91, 79, ..., 91, 79, 79],
         [67, 67, 67, ..., 67, 67, 67],
         ...,
         [19, 19, 11, ..., 47, 47, 55],
         [19, 27, 19, ..., 47, 47, 47],
         [11, 19, 19, ..., 27, 19, 19]]], dtype=uint8),
 -4.0,
 False,
 array([50.]))

In [30]:
env.step(2)

(array([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],
 
        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]),
 99.0,
 True,
 0)

In [31]:
env.close()

### Environment done, can train now

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common import env_checker
import os 
# Import callback class from sb3
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True


CHECKPOINT_DIR = './train/train_basic'
LOG_DIR = './logs/log_basic'

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
env = VizDoomEnv("ViZDoom/scenarios/basic.cfg", render=False)
model = PPO("CnnPolicy",env,tensorboard_log=logs,device="cpu",verbose=1,learning_rate=0.0001,n_steps=256)
# stable baselines documentation says that ppo runs better on cpu
model.learn(total_timesteps=100000, callback=callback)
#tensorboard --logdir =. 


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
